In [1]:
#Notebook description

#This notebook is being used to evaluate the techinical market conditions of a single asset and assess
#the appropriate strategy to take in order to maximize returns.

In [2]:
#take all compuation functions and put them in a separate file

#simplify the date x axis on the percent drawdown chart

#default the zoom range to a comfortable range, and create a dropdown to select the time range for Volatility section

#properly label and annoate the garch models

#Remove the VIX charting, its redudnant now that we have the volatility models

In [3]:
#Load libraries 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True
logger.propagate = False
# Load libraries
from Quantapp.Plotter import Plotter
from Quantapp.Computation import Computation
from Quantapp.EconomicData import EconomicData

import numpy as np
import json
import os
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
from plotly.subplots import make_subplots
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#shut down warnings
import warnings
warnings.filterwarnings("ignore")


qc = Computation()
qp = Plotter()
qe = EconomicData()

def simplify_datetime_index(series):
    """
    Simplifies the DateTime index of a Series to contain only the date (YYYY-MM-DD),
    maintaining it as a DateTimeIndex without timezone information.
    
    Parameters:
        series (pd.Series): The input Series with a DateTimeIndex.
    
    Returns:
        pd.Series: The Series with the DateTime index simplified to YYYY-MM-DD.
    """
    if not isinstance(series.index, pd.DatetimeIndex):
        raise TypeError("The Series index must be a DateTimeIndex.")
    
    # Remove timezone information if present
    if series.index.tz is not None:
        series = series.copy()
        series.index = series.index.tz_convert('UTC').tz_localize(None)
    
    # Normalize the index to remove the time component
    series.index = series.index.normalize()
    
    return series
                

In [4]:
#PARAMETERS

time_frame_week = 7
time_frame_short = 21
time_frame_mid   = 50
time_frame_long = 200

interval = '1d'
period     = '10y'
risk_free_rate = 0.02 / 252  # Annualized risk-free rate divided by trading days

#should be a string or None
ticker_str = 'SPY'



In [5]:
#Load data

ticker = yf.Ticker(ticker_str).history(period=period, interval=interval)
ticker = simplify_datetime_index(ticker)


#retreive options data from yahoo finance
options = yf.Ticker(ticker_str).options
options = pd.DataFrame(options, columns=['Expiration Date'])

#add another column called date till expiration
options['Date Till Expiration'] = pd.to_datetime(options['Expiration Date']) - pd.to_datetime('today')
options['Date Till Expiration'] = options['Date Till Expiration'].dt.days


#retreive the contracts for the first expiration date and display them as a dataframe
contracts = yf.Ticker(ticker_str).option_chain(options['Expiration Date'][0])
call_contract_chain = pd.DataFrame(contracts.calls)
put_call_contract_chain = pd.DataFrame(contracts.puts)
underlying_data = contracts.underlying


columns_to_drop = ['currency', 'contractSize', 'percentChange', 'change']

call_contract_chain = call_contract_chain.drop(columns=columns_to_drop)
put_call_contract_chain = put_call_contract_chain.drop(columns=columns_to_drop)
display(underlying_data)

{'language': 'en-US',
 'region': 'US',
 'quoteType': 'ETF',
 'typeDisp': 'ETF',
 'quoteSourceName': 'Nasdaq Real Time Price',
 'triggerable': True,
 'customPriceAlertConfidence': 'HIGH',
 'currency': 'USD',
 'corporateActions': [],
 'postMarketTime': 1754092799,
 'regularMarketTime': 1754078400,
 'exchange': 'PCX',
 'marketState': 'CLOSED',
 'tradeable': False,
 'cryptoTradeable': False,
 'hasPrePostMarketData': True,
 'firstTradeDateMilliseconds': 728317800000,
 'priceHint': 2,
 'postMarketChangePercent': -0.0804221,
 'postMarketPrice': 621.22,
 'postMarketChange': -0.5,
 'regularMarketChange': -10.36,
 'regularMarketDayHigh': 626.34,
 'regularMarketDayRange': '619.29 - 626.34',
 'regularMarketDayLow': 619.29,
 'regularMarketVolume': 139492478,
 'regularMarketPreviousClose': 632.08,
 'bid': 621.98,
 'ask': 621.23,
 'bidSize': 7,
 'askSize': 3,
 'fullExchangeName': 'NYSEArca',
 'financialCurrency': 'USD',
 'regularMarketOpen': 626.3,
 'averageDailyVolume3Month': 70862608,
 'averageDail

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Get the current stock price from underlying data
current_price = underlying_data['regularMarketPrice']

# Create a function to filter and display options based on strike range
def show_options_table(strikes_to_show):
    # Calculate the strike range
    half_range = strikes_to_show // 2
    min_strike = current_price - half_range
    max_strike = current_price + half_range
    
    # Filter call options
    filtered_calls = call_contract_chain[
        (call_contract_chain['strike'] >= min_strike) & 
        (call_contract_chain['strike'] <= max_strike)
    ]
    
    # Filter put options
    filtered_puts = put_call_contract_chain[
        (put_call_contract_chain['strike'] >= min_strike) & 
        (put_call_contract_chain['strike'] <= max_strike)
    ]
    
    # Create a figure with two side-by-side tables
    fig = make_subplots(
        rows=1, cols=2,
        column_widths=[0.5, 0.5],
        subplot_titles=("Call Options", "Put Options"),
        specs=[[{"type": "table"}, {"type": "table"}]]
    )

    # Get expiration date from the options DataFrame
    expiration_date = options['Expiration Date'][0]
    
    # Create conditional fill colors for calls based on inTheMoney status
    call_fill_colors = []
    for itm in filtered_calls['inTheMoney']:
        if itm:
            call_fill_colors.append('rgba(144, 238, 144, 0.6)')  # Light green for in-the-money
        else:
            call_fill_colors.append('lavender')  # Default for out-of-the-money
    
    # Add call options table with conditional formatting
    fig.add_trace(
        go.Table(
            header=dict(values=list(filtered_calls.columns),
                        fill_color='paleturquoise',
                        align='left',
                        font=dict(size=11)),
            cells=dict(values=[filtered_calls[col] for col in filtered_calls.columns],
                       fill_color=[call_fill_colors for _ in filtered_calls.columns],
                       align='left')
        ),
        row=1, col=1
    )
    
    # Create conditional fill colors for puts based on inTheMoney status
    put_fill_colors = []
    for itm in filtered_puts['inTheMoney']:
        if itm:
            put_fill_colors.append('rgba(144, 238, 144, 0.6)')  # Light green for in-the-money
        else:
            put_fill_colors.append('mistyrose')  # Default for out-of-the-money
    
    # Add put options table with conditional formatting
    fig.add_trace(
        go.Table(
            header=dict(values=list(filtered_puts.columns),
                        fill_color='lightpink',
                        align='left'),
            cells=dict(values=[filtered_puts[col] for col in filtered_puts.columns],
                       fill_color=[put_fill_colors for _ in filtered_puts.columns],
                       align='left')
        ),
        row=1, col=2
    )
    
    # Update layout with title including the range information
    fig.update_layout(
        title_text=f"Option Chain for {ticker_str} - Expiration: {expiration_date}<br>" + 
                  f"<span style='font-size:12px;'>Green = In The Money | " +
                  f"Showing strikes from ${min_strike:.1f} to ${max_strike:.1f}</span>",
        height=800,
        autosize=True,
    )
    
    # Display with config that enables responsiveness
    fig.show(config={'responsive': True, 'scrollZoom': True})

# Create dropdown for strike selection
strike_dropdown = widgets.Dropdown(
    options=[(f"{i} Strikes", i) for i in [10, 20, 30, 40, 50, 60]],
    value=20,
    description='Show:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

# Function to handle dropdown change
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)
        display(strike_dropdown)
        show_options_table(change['new'])

# Register the callback
strike_dropdown.observe(on_change, names='value')

# Initial display
display(strike_dropdown)
show_options_table(strike_dropdown.value)


Dropdown(description='Show:', index=1, layout=Layout(width='200px'), options=(('10 Strikes', 10), ('20 Strikes…